In [11]:
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re
import os
import sys
import glob
import shutil
from pathlib import Path

In [2]:
# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT
# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
driver.implicitly_wait(0)

C:\Users\ntthu\AppData\Local\Temp\ipykernel_15480\1445639418.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)


## Thu thập dữ liệu

In [77]:
def dulieu(url):
    driver.get(url)
    data=[]
    # for i in range(1):
    while True:
        sleep(5)
        element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="Review"]')))
        html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
        soup=BeautifulSoup(html_of_interest, 'html.parser')
        # l=soup.select('div[class="Review-comment"]')
        content = soup.select('div[class="Review-comments"]')[0]
        content=content.select('div[class="Review-comment"]')
        for nd in content:
            nd_dict={}
            trai=nd.select('div[class="Review-comment-left"]')[0]
            diem=trai.select('div[class="Review-comment-leftScore"]')[0].text
            nd_dict['Điểm đánh giá'] = diem
            ten=trai.select('div[data-info-type="reviewer-name"]')
            if ten == []:
                nd_dict['Đến từ']=''
            else:
                nd_dict['Đến từ']=ten[0].text
            group=trai.select('div[data-info-type="group-name"]')
            if group == []:
                nd_dict['Đi theo']=''
            else:
                nd_dict['Đi theo']=group[0].text 
            phong=trai.select('div[data-info-type="room-type"]')
            if phong == []:
                nd_dict['Kiểu phòng']=''
            else:
                nd_dict['Kiểu phòng']=phong[0].text
            detail=trai.select('div[data-info-type="stay-detail"]')
            if detail == []:
                nd_dict['Số ngày']=''
            else:
                nd_dict['Số ngày']=detail[0].text
            phai=nd.select('div[class="Review-comment-right"]')[0]
            phai=phai.select('div[class="Review-comment-body"]')[0]
            tieude=phai.select('h3[class="Review-comment-bodyTitle"]')
            if tieude == []:
                nd_dict['Title']=''
            else:
                nd_dict['Title']=tieude[0].text
            positive=phai.select('div[data-type="positive"]')
            if positive == []:
                nx=''
            else:
                nx=positive[0].text
            comment=phai.select('p[data-type="comment"]')
            if comment == []:
                nx=nx+''
            else:
                nx=nx+'\n'+comment[0].text
            nd_dict['Nhận xét']=nx
            # print(nd_dict)
            data.append(nd_dict)
        c=soup.select('div[class="Review-paginator-steps"]')[0]
        chuyen=c.select('span[aria-label="Trang đánh giá kế tiếp"]')[0]
        # print(chuyen)
        x=re.findall('<span aria-disabled=(.*?) aria-label=.*?>.*?<\/span>', str(chuyen))
        if x[0]=='"true"':
            break
        driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'i[class="ficon ficon-24 ficon-carrouselarrow-right"]'))))
    return data

In [78]:
urls = ['https://www.agoda.com/vi-vn/muong-thanh-grand-da-nang-hotel/hotel/da-nang-vn.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1844104&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-05-2&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=VND&isFreeOccSearch=false&isCityHaveAsq=false&los=1&searchrequestid=0436ad18-0f72-4f88-82b8-daba39cf777f#reviewSection',
       'https://www.agoda.com/vi-vn/oneopera-danang-hotel/hotel/da-nang-vn.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1844104&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-05-2&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=VND&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=5&los=1&searchrequestid=0436ad18-0f72-4f88-82b8-daba39cf777f',
       'https://www.agoda.com/vi-vn/muong-thanh-luxury-da-nang-hotel/hotel/da-nang-vn.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1844104&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-05-2&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=VND&isFreeOccSearch=false&isCityHaveAsq=false&los=1&searchrequestid=43de9d5b-8a3d-472e-bf5a-fe1a0ff35c71']
grand = pd.DataFrame(dulieu(urls[0]))
grand.to_excel(r'grand.xlsx', index=False)
oneopera = pd.DataFrame(dulieu(urls[1]))
oneopera.to_excel(r'oneopera.xlsx', index=False)
luxury = pd.DataFrame(dulieu(urls[2]))
luxury.to_excel(r'luxury.xlsx', index=False)

In [79]:
grand['Tên khách sạn']='Khách sạn Mường Thanh Grand Đà Nẵng (Muong Thanh Grand Da Nang Hotel)'
grand['Địa chỉ'] = '962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà Nẵng, Việt Nam'
grand

,Điểm đánh giá,Đến từ,Đi theo,Kiểu phòng,Số ngày,Title,Nhận xét,Tên khách sạn,Địa chỉ
0,"8,4",Ngoc từ Việt Nam,Nhóm,Phòng Deluxe 2 Giường đơn,Đã ở 1 đêm vào tháng chín 2015,"Vị trí tốt, phù hợp công tác hay du lịch”","\nKhách sạn nằm vị trí gần cầu Rồng, giá cả tố...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
1,"9,0",Thuy từ Việt Nam,Cặp đôi,,Đã ở 4 đêm vào tháng mười hai 2013,"Dịch vụ tốt, nhân viên thân thiện, phòng tiện ...","Dịch vụ tốt, nhân viên thân thiện, phòng tiện ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
2,"9,7",SY từ Việt Nam,Gia đình có em bé,,Đã ở 5 đêm vào tháng mười một 2013,"Khách sạn sang trọng tiện nghi, vị trí thuận t...","Phòng đẹp, tiện nghi, nhân viên thân thiện\nKỳ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
3,"8,3",Quang từ Việt Nam,Gia đình có em bé,Executive Suite,Đã ở 4 đêm vào tháng bảy 2015,Khách sạn tốt!”,1) Phòng nghỉ tốt. 2) Giá hợp lý. 3) Phục vụ t...,Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
4,"9,7",Nam từ Việt Nam,Gia đình có em bé,Deluxe gia đình 3 giường,Đã ở 2 đêm vào tháng một 2015,Phòng tốt và sạch sẽ”,"Phòng sạch sẽ, giường ngủ rộng,\nPhòng sạch sẽ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
...,...,...,...,...,...,...,...,...,...
771,"7,7",Yang từ Trung Quốc,Cặp đôi,,Đã ở 1 đêm vào tháng một 2014,房間漂亮乾淨”,房屋裝潢很棒，飯店整體整潔乾淨，但看不到河景比較可惜，早餐菜色選擇較少。整體以$60來說算是...,Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
772,"8,3",HAN-SHENG từ Đài Loan,Nhóm,Superior 2 giường,Đã ở 2 đêm vào tháng mười hai 2013,性價比很高的飯店”,早餐豐富 游泳池很好 房間整潔,Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
773,"10,0",HAN-SHENG từ Đài Loan,Nhóm,,Đã ở 3 đêm vào tháng mười hai 2013,性價比很高”,早餐豐富 服務人員熱情 房間整潔,Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
774,"9,0",Hsia từ Đài Loan,Đi công tác,,Đã ở 2 đêm vào tháng mười một 2013,房間大!工作人員友善!值得再去住!”,"房間大!清潔!工作人員友善!\n非常好的酒店!樓層夠高!面將風景優美開窗又大,餐廳素質是可以...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."


In [80]:
oneopera['Tên khách sạn']='Khách Sạn Mường Thanh Luxury Sông Hàn (Muong Thanh Luxury Song Han Hotel)'
oneopera['Địa chỉ'] = '115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hải Châu , Thành Phố Đà Nẵng, Việt Nam'
oneopera

,Điểm đánh giá,Đến từ,Đi theo,Kiểu phòng,Số ngày,Title,Nhận xét,Tên khách sạn,Địa chỉ
0,"9,3",Phuong từ Việt Nam,Du lịch một mình,Superior 2 giường Hướng phố,Đã ở 1 đêm vào tháng ba 2010,HAGL Plaza sang trọng”,"Khách sạn nằm ở vị trí đẹp, thuận tiện cho việ...",Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
1,"9,0",HOANG từ Việt Nam,Gia đình có trẻ em,Superior 2 giường Hướng phố,Đã ở 3 đêm vào tháng bảy 2013,Rất hài lòng về cung cách phục vụ và phòng ốc.”,"Phòng rộng, nhân viên vui vẻ, giá cả hợp lý.\n...",Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
2,"9,2",Park,Gia đình có em bé,Phòng Deluxe King,Đã ở 1 đêm vào tháng sáu 2019,"Khách sạn gần sân bay, trung tâm ăn uống và ph...",\nChúng tôi đã có một đêm nghỉ tuyệt vời tại đ...,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
3,"9,2",Ánh từ Việt Nam,Cặp đôi,Superior 2 giường Hướng phố,Đã ở 1 đêm vào tháng tư 2018,đáng để trở lại”,\nmọi thứ đều rất tốt và chuyên nghiệp. Mình đ...,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
4,"10,0",Dieu,Gia đình có trẻ em,Phòng Deluxe King,Đã ở 1 đêm vào tháng sáu 2019,Khách sạn khá ổn”,"\nKhách sạn sạch sẽ, nhân viên thân thiện. Tần...",Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
...,...,...,...,...,...,...,...,...,...
625,"4,0",Công từ Việt Nam,Cặp đôi,Phòng Hai Giường Đơn Loại Sang,Đã ở 1 đêm vào tháng bảy 2022,Tot”,\nTot,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
626,"9,6",Binh từ Việt Nam,Du lịch một mình,Phòng Deluxe King,Đã ở 1 đêm vào tháng sáu 2022,Tot”,\nTot,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
627,"9,6",Thuy từ Việt Nam,Nhóm,Superior 2 giường Hướng phố,Đã ở 2 đêm vào tháng ba 2023,Trên cả tuyệt vời”,\nOk,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."
628,"8,8",Por từ Singapore,Cặp đôi,Superior 2 giường Hướng phố,Đã ở 2 đêm vào tháng mười một 2022,Tuyệt vời”,\nPleasant stay.,Khách Sạn Mường Thanh Luxury Sông Hàn (Muong T...,"115 Nguyễn Văn Linh, Phường Nam Dương, Quận Hả..."


In [81]:
luxury['Tên khách sạn']='Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Thanh Luxury Da Nang hotel)'
luxury['Địa chỉ'] = '270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hành Sơn, Thành Phố Đà Nẵng, Phước Mỹ, Việt Nam'
luxury

,Điểm đánh giá,Đến từ,Đi theo,Kiểu phòng,Số ngày,Title,Nhận xét,Tên khách sạn,Địa chỉ
0,"8,8",Đặng từ Việt Nam,Gia đình có em bé,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng bảy 2018,"Khách sạn dành cho người năng động, sôi nổi”","\nKhách sạn sạch sẽ, view đẹp, nhân viên thân...",Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
1,"8,8",Thuy từ Việt Nam,Gia đình có trẻ em,,Đã ở 1 đêm vào tháng năm 2018,Tuyệt vời”,\nKhách sạn ngay mặt tiền biển nên thuận lợi c...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
2,"8,0",Khánh,Gia đình có em bé,Deluxe giường King Hướng biển,Đã ở 3 đêm vào tháng tư 2019,Nhân viên tệ”,\nTôi đi du lịch cùng gia đình. Lúc vào checki...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
3,"10,0",Ha từ Việt Nam,Gia đình có em bé,Phòng Superior 2 giường không có cửa sổ,Đã ở 2 đêm vào tháng sáu 2020,Tuyệt vời”,\nTôi đã có 1 kỳ nghỉ tuyệt vời cùng gia đình ...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
4,"10,0",TIEN từ Việt Nam,Gia đình có trẻ em,Deluxe giường King,Đã ở 4 đêm vào tháng bảy 2019,Trên cả tuyệt vời”,"\nKhách sạn ngay ven biển, bãi tắm rất đẹp, ăn...",Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
...,...,...,...,...,...,...,...,...,...
993,"2,8",yusheng từ Đài Loan,Du lịch một mình,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng tư 2018,品質待改進”,\n1.游泳池池水品質待改進\n2.未提供腳踏車出租\n3.4/30焰火自助餐沒有提醒住客參...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
994,"7,6",CHIA-CHUN từ Campuchia,Cặp đôi,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng tư 2018,無敵超級海景”,\n升等的感覺真好,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
995,"9,2",Phoebe từ Đài Loan,Cặp đôi,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng mười 2017,無敵海景的好飯店”,\n一進房發現房間超級無敵大，二大面牆都是玻璃，海景一覽無遺，飯店將我們升等至最大房間，景色...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
996,"8,0",Ha từ Việt Nam,Cặp đôi,Deluxe giường King,Đã ở 1 đêm vào tháng hai 2023,Tot”,\nTot,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."


### Dữ liệu sau khi thu thập

In [229]:
data=pd.concat([grand, oneopera, luxury])
data.rename(columns = {'Đến từ':'Quốc gia','Đi theo': 'Kiểu khách hàng','Số ngày': 'Thời gian'}, inplace = True)
data.replace('', np.NaN,inplace=True)
# data.to_excel(r'ThuThapDuLieu.xlsx', index=False)
data

,Điểm đánh giá,Quốc gia,Kiểu khách hàng,Kiểu phòng,Thời gian,Title,Nhận xét,Tên khách sạn,Địa chỉ
0,"8,4",Ngoc từ Việt Nam,Nhóm,Phòng Deluxe 2 Giường đơn,Đã ở 1 đêm vào tháng chín 2015,"Vị trí tốt, phù hợp công tác hay du lịch”","\nKhách sạn nằm vị trí gần cầu Rồng, giá cả tố...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
1,"9,0",Thuy từ Việt Nam,Cặp đôi,NaN,Đã ở 4 đêm vào tháng mười hai 2013,"Dịch vụ tốt, nhân viên thân thiện, phòng tiện ...","Dịch vụ tốt, nhân viên thân thiện, phòng tiện ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
2,"9,7",SY từ Việt Nam,Gia đình có em bé,NaN,Đã ở 5 đêm vào tháng mười một 2013,"Khách sạn sang trọng tiện nghi, vị trí thuận t...","Phòng đẹp, tiện nghi, nhân viên thân thiện\nKỳ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
3,"8,3",Quang từ Việt Nam,Gia đình có em bé,Executive Suite,Đã ở 4 đêm vào tháng bảy 2015,Khách sạn tốt!”,1) Phòng nghỉ tốt. 2) Giá hợp lý. 3) Phục vụ t...,Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
4,"9,7",Nam từ Việt Nam,Gia đình có em bé,Deluxe gia đình 3 giường,Đã ở 2 đêm vào tháng một 2015,Phòng tốt và sạch sẽ”,"Phòng sạch sẽ, giường ngủ rộng,\nPhòng sạch sẽ...",Khách sạn Mường Thanh Grand Đà Nẵng (Muong Tha...,"962 Ngô Quyền, Quận Sơn Trà, Thành Phố Đà N..."
...,...,...,...,...,...,...,...,...,...
993,"2,8",yusheng từ Đài Loan,Du lịch một mình,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng tư 2018,品質待改進”,\n1.游泳池池水品質待改進\n2.未提供腳踏車出租\n3.4/30焰火自助餐沒有提醒住客參...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
994,"7,6",CHIA-CHUN từ Campuchia,Cặp đôi,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng tư 2018,無敵超級海景”,\n升等的感覺真好,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
995,"9,2",Phoebe từ Đài Loan,Cặp đôi,Deluxe giường King Hướng biển,Đã ở 2 đêm vào tháng mười 2017,無敵海景的好飯店”,\n一進房發現房間超級無敵大，二大面牆都是玻璃，海景一覽無遺，飯店將我們升等至最大房間，景色...,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
996,"8,0",Ha từ Việt Nam,Cặp đôi,Deluxe giường King,Đã ở 1 đêm vào tháng hai 2023,Tot”,\nTot,Khách sạn Mường Thanh Luxury Đà Nẵng (Muong Th...,"270 Võ Nguyên Giáp, Phường Mỹ An, Quận Ngũ Hàn..."
